In [5]:
!pip install scikit-
!pip install scikit-optimize

ERROR: Invalid requirement: 'scikit-': Expected end or semicolon (after name and no valid version specifier)
    scikit-
          ^


In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.calibration import CalibratedClassifierCV


from sklearn.model_selection import (
    cross_validate,
    StratifiedKFold,
    GridSearchCV,
    RandomizedSearchCV
)
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    classification_report,
    confusion_matrix,
    roc_curve,
    roc_auc_score
)
# Estadísticas
from scipy import stats
from scipy.stats import f_oneway
import itertools
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd


##CARGAR DATOS

In [11]:
df_train = pd.read_csv("heart_disease_train.csv")
X_train = df_train.drop('HeartDisease', axis=1)
y_train = df_train['HeartDisease'].map({"No": 0, "Yes": 1})

df_test = pd.read_csv("heart_disease_test.csv")
X_test = df_test.drop('HeartDisease', axis=1)
y_test = df_test['HeartDisease'].map({"No": 0, "Yes": 1})

print(f" Conjunto de entrenamiento: {X_train.shape[0]} muestras, {X_train.shape[1]} características")
print(f"Conjunto de prueba: {X_test.shape[0]} muestras")
print(f"\nDistribución de clases en entrenamiento:")
print(df_train['HeartDisease'].value_counts())

 Conjunto de entrenamiento: 409390 muestras, 17 características
Conjunto de prueba: 95939 muestras

Distribución de clases en entrenamiento:
HeartDisease
No     204695
Yes    204695
Name: count, dtype: int64



# Definir columnas y preprocesador


In [17]:
numerical_cols = ['BMI', 'PhysicalHealth', 'SleepTime', 'MentalHealth']

categorical_cols = [
    'Smoking', 'AlcoholDrinking', 'Stroke', 'DiffWalking', 'Sex',
    'AgeCategory', 'Race', 'Diabetic', 'PhysicalActivity',
    'GenHealth', 'Asthma', 'KidneyDisease', 'SkinCancer'
]

# Cambio clave: sparse_output=False → genera matriz densa (necesario para GaussianNB)
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', drop='first', sparse_output=False), categorical_cols)
    ],
    remainder='passthrough'
)


##DEFINIR MODELOS BASE (SIN GRID SEARCH)

In [18]:
# Modelos sin optimización, con preprocesamiento incluido
base_models = {
    'Decision Tree': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', DecisionTreeClassifier(
            criterion='gini',
            max_depth=5,
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=42
        ))
    ]),

    'Regresión Logística': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', LogisticRegression(
            max_iter=1000,
            random_state=42
        ))
    ]),

    'Naive Bayes': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', GaussianNB())
    ]),

    'Random Forest': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=42,
            n_jobs=-1
        ))
    ]),

    'Gradient Boosting': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', GradientBoostingClassifier(
            n_estimators=100,
            learning_rate=0.1,
            max_depth=5,
            min_samples_split=10,
            min_samples_leaf=5,
            random_state=42
        ))
    ]),

    'AdaBoost': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', AdaBoostClassifier(
            estimator=DecisionTreeClassifier(max_depth=3),
            n_estimators=50,
            learning_rate=1.0,
            random_state=42
        ))
    ]),

    'XGBoost': Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', XGBClassifier(
            n_estimators=100,
            max_depth=6,
            learning_rate=0.1,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42,
            n_jobs=-1,
            eval_metric='logloss'
        ))
    ])
}

print("Modelos listos para entrenamiento con pipeline completo:")
for name in base_models.keys():
    print(name)

Modelos listos para entrenamiento con pipeline completo:
Decision Tree
Regresión Logística
Naive Bayes
Random Forest
Gradient Boosting
AdaBoost
XGBoost


##VALIDACIÓN CRUZADA - MODELOS BASE

In [19]:
cv_folds = 3
kfold = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)

scoring = {
    'accuracy': 'accuracy',
    'precision': 'precision',
    'recall': 'recall',
    'f1': 'f1',
    'roc_auc': 'roc_auc'
}

cv_results_list = []

print("\nEvaluando modelos base...\n")

for model_name, model in base_models.items():
    print(f"Evaluando {model_name}...")

    scores = cross_validate(
        model, X_train, y_train,
        cv=kfold,
        scoring=scoring,
        n_jobs=-1,
        return_train_score=False,
        error_score='raise'
    )

    cv_results_list.append({
        'Modelo': model_name,
        'Accuracy_CV': scores['test_accuracy'].mean(),
        'Precision_CV': scores['test_precision'].mean(),
        'Recall_CV': scores['test_recall'].mean(),
        'F1_CV': scores['test_f1'].mean(),
        'ROC_AUC_CV': scores['test_roc_auc'].mean()
    })

    print(f"  F1: {scores['test_f1'].mean():.4f} | Recall: {scores['test_recall'].mean():.4f} | Precision: {scores['test_precision'].mean():.4f} | Accuracy: {scores['test_accuracy'].mean():.4f}")


Evaluando modelos base...

Evaluando Decision Tree...
  F1: 0.7294 | Recall: 0.7601 | Precision: 0.7011 | Accuracy: 0.7180
Evaluando Regresión Logística...
  F1: 0.7694 | Recall: 0.7872 | Precision: 0.7524 | Accuracy: 0.7640
Evaluando Naive Bayes...
  F1: 0.7492 | Recall: 0.8973 | Precision: 0.6430 | Accuracy: 0.6996
Evaluando Random Forest...
  F1: 0.7659 | Recall: 0.7679 | Precision: 0.7638 | Accuracy: 0.7653
Evaluando Gradient Boosting...
  F1: 0.8333 | Recall: 0.8312 | Precision: 0.8355 | Accuracy: 0.8338
Evaluando AdaBoost...
  F1: 0.8159 | Recall: 0.8058 | Precision: 0.8264 | Accuracy: 0.8182
Evaluando XGBoost...
  F1: 0.8334 | Recall: 0.8350 | Precision: 0.8318 | Accuracy: 0.8331


##Anova y Toukey

In [20]:
# Recolectar todos los scores de F1 por fold para cada modelo
cv_folds = 3
kfold = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)

# Diccionario para almacenar scores por modelo
scores_by_model = {}

for model_name, model in base_models.items():
    print(f"  Evaluando {model_name}...")

    #Usar diccionario en scoring
    scores = cross_validate(
        model, X_train, y_train,
        cv=kfold,
        scoring={'f1': 'f1'},
        n_jobs=-1,
        return_train_score=False
    )

    scores_by_model[model_name] = scores['test_f1']

# Crear DataFrame para análisis

# Mostrar estadísticas descriptivas
print("ESTADÍSTICAS DESCRIPTIVAS (F1-Score por modelo):")
print("-" * 100)
stats_df = pd.DataFrame({
    'Modelo': list(scores_by_model.keys()),
    'Media': [np.mean(scores) for scores in scores_by_model.values()],
    'Std': [np.std(scores) for scores in scores_by_model.values()],
    'Min': [np.min(scores) for scores in scores_by_model.values()],
    'Max': [np.max(scores) for scores in scores_by_model.values()]
})
stats_df = stats_df.sort_values('Media', ascending=False)
print(stats_df.to_string(index=False, float_format='%.4f'))

# TEST ANOVA

# Preparar datos para ANOVA
model_scores = list(scores_by_model.values())

# Realizar ANOVA
f_statistic, p_value = f_oneway(*model_scores)

print(f"\nEstadístico F: {f_statistic:.4f}")
print(f"P-value: {p_value:.6f}")

if p_value < 0.05:
    print("\n Resultado: Hay diferencias SIGNIFICATIVAS entre los modelos (p < 0.05)")
    print("  → Procederemos con el Test de Tukey para comparaciones múltiples")
else:
    print("\n Resultado: NO hay diferencias significativas entre los modelos (p >= 0.05)")
    print("  Los modelos tienen rendimientos estadísticamente similares")

# TEST DE TUKEY (Post-hoc)
print("TEST DE TUKEY (Honestly Significant Difference)")


# Crear DataFrame en formato largo para Tukey
tukey_data = []
for model_name, scores in scores_by_model.items():
    for score in scores:
        tukey_data.append({'Modelo': model_name, 'F1_Score': score})

tukey_df = pd.DataFrame(tukey_data)

# Instalar scikit-posthocs si no está instalado
try:
    from scikit_posthocs import posthoc_tukey

    # Realizar Test de Tukey
    tukey_result = posthoc_tukey(tukey_df, val_col='F1_Score', group_col='Modelo')

    print("\nMatriz de p-values del Test de Tukey:")
    print("(Valores < 0.05 indican diferencias significativas entre modelos)")
    print("-" * 100)
    print(tukey_result.to_string(float_format='%.4f'))

except ImportError:
    print("\n Advertencia: scikit-posthocs no está instalado")
    print("Instalando: pip install scikit-posthocs")
    print("\nRealizando comparaciones pareadas manualmente con t-test...")

    # Alternativa: t-test pareado manual
    from scipy.stats import ttest_ind

    model_names = list(scores_by_model.keys())
    n_models = len(model_names)

    # Crear matriz de p-values
    p_matrix = np.ones((n_models, n_models))

    for i, model1 in enumerate(model_names):
        for j, model2 in enumerate(model_names):
            if i != j:
                t_stat, p_val = ttest_ind(scores_by_model[model1],
                                         scores_by_model[model2])
                p_matrix[i, j] = p_val

    tukey_result = pd.DataFrame(p_matrix,
                               index=model_names,
                               columns=model_names)

    print("\nMatriz de p-values (t-test pareado):")
    print("(Valores < 0.05 indican diferencias significativas)")
    print("-" * 100)
    print(tukey_result.to_string(float_format='%.4f'))

# SELECCIÓN DE LOS 3 MEJORES MODELOS

print("SELECCIÓN DE LOS 3 MEJORES MODELOS")

# Ordenar modelos por media de F1-Score
top_3_models = stats_df.head(3)['Modelo'].tolist()

print("\n TOP 3 MODELOS SELECCIONADOS (basado en F1-Score medio):")
for i, model_name in enumerate(top_3_models, 1):
    mean_f1 = stats_df[stats_df['Modelo'] == model_name]['Media'].values[0]
    std_f1 = stats_df[stats_df['Modelo'] == model_name]['Std'].values[0]
    print(f"{i}. {model_name:<25} F1-Score: {mean_f1:.4f} (±{std_f1:.4f})")

# Verificar si hay diferencias significativas entre el top 3
print("\n ANÁLISIS DE DIFERENCIAS ENTRE TOP 3:")

for i in range(len(top_3_models)):
    for j in range(i+1, len(top_3_models)):
        model1 = top_3_models[i]
        model2 = top_3_models[j]
        p_val = tukey_result.loc[model1, model2]

        if p_val < 0.05:
            print(f"  • {model1} vs {model2}: p={p_val:.4f} → Diferencia SIGNIFICATIVA")
        else:
            print(f"  • {model1} vs {model2}: p={p_val:.4f} → Sin diferencia significativa")


print("ESTOS 3 MODELOS PASARÁN AL PROCESO DE OPTIMIZACIÓN (GRID SEARCH)")

# Guardar los nombres de los top 3 para usar después
selected_models = {name: base_models[name] for name in top_3_models}

  Evaluando Decision Tree...
  Evaluando Regresión Logística...
  Evaluando Naive Bayes...
  Evaluando Random Forest...
  Evaluando Gradient Boosting...
  Evaluando AdaBoost...
  Evaluando XGBoost...
ESTADÍSTICAS DESCRIPTIVAS (F1-Score por modelo):
----------------------------------------------------------------------------------------------------
             Modelo  Media    Std    Min    Max
            XGBoost 0.8334 0.0004 0.8330 0.8340
  Gradient Boosting 0.8333 0.0011 0.8319 0.8345
           AdaBoost 0.8159 0.0025 0.8128 0.8189
Regresión Logística 0.7694 0.0006 0.7688 0.7701
      Random Forest 0.7659 0.0013 0.7642 0.7675
        Naive Bayes 0.7492 0.0004 0.7489 0.7498
      Decision Tree 0.7294 0.0013 0.7276 0.7305

Estadístico F: 2161.2385
P-value: 0.000000

 Resultado: Hay diferencias SIGNIFICATIVAS entre los modelos (p < 0.05)
  → Procederemos con el Test de Tukey para comparaciones múltiples
TEST DE TUKEY (Honestly Significant Difference)

 Advertencia: scikit-posthocs no 

##OPTIMIZACIÓN DE LOS 3 MEJORES MODELOS (GRID SEARCH)

In [26]:
import time
param_grids = {
    'XGBoost': {
        'classifier__n_estimators': [100],
        'classifier__max_depth': [4, 6],
        'classifier__learning_rate': [0.1],
        'classifier__subsample': [0.8]
    },
    'Gradient Boosting': {
        'classifier__n_estimators': [100],
        'classifier__learning_rate': [0.1],
        'classifier__max_depth': [3]
    },
    'AdaBoost': {
        'classifier__n_estimators': [50],
        'classifier__learning_rate': [1.0]
    }
}

best_models = {}

for model_name, model in selected_models.items():
    print(f" Optimizando {model_name}...")

    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grids[model_name],
        scoring='f1',
        cv=2,
        n_jobs=-1,
        verbose=1
    )

    start_time = time.time()
    grid_search.fit(X_train, y_train)
    elapsed = time.time() - start_time

    best_models[model_name] = grid_search.best_estimator_

    print(f" Mejor F1-score (CV): {grid_search.best_score_:.4f}")
    print(f" Mejores hiperparámetros: {grid_search.best_params_}")
    print(f" Tiempo total: {elapsed/60:.2f} minutos")

print("Optimización express completada.")



 Optimizando XGBoost...
Fitting 2 folds for each of 2 candidates, totalling 4 fits
 Mejor F1-score (CV): 0.8274
 Mejores hiperparámetros: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 6, 'classifier__n_estimators': 100, 'classifier__subsample': 0.8}
 Tiempo total: 0.55 minutos
 Optimizando Gradient Boosting...
Fitting 2 folds for each of 1 candidates, totalling 2 fits
 Mejor F1-score (CV): 0.8067
 Mejores hiperparámetros: {'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__n_estimators': 100}
 Tiempo total: 4.56 minutos
 Optimizando AdaBoost...
Fitting 2 folds for each of 1 candidates, totalling 2 fits
 Mejor F1-score (CV): 0.8079
 Mejores hiperparámetros: {'classifier__learning_rate': 1.0, 'classifier__n_estimators': 50}
 Tiempo total: 2.60 minutos
Optimización express completada.


In [34]:
import joblib
# Usar 10% del dataset de entrenamiento para optimización rápida
X_small = X_train.sample(frac=0.1, random_state=42)
y_small = y_train.loc[X_small.index]
print(f"Usando muestra de {len(X_small)} registros para optimización rápida.\n")

# Parámetros de control
N_ITER_RS = 5   # número de combinaciones aleatorias
CV = 2          # número de folds
RANDOM_STATE = 42

# -----------------------------
# Definir los estimadores base (sin pipeline interno)
# -----------------------------
from xgboost import XGBClassifier
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

estimators_base = {
    'XGBoost': XGBClassifier(
        n_estimators=100,
        max_depth=6,
        learning_rate=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=RANDOM_STATE,
        n_jobs=-1,
        eval_metric='logloss'
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        random_state=RANDOM_STATE
    ),
    'AdaBoost': AdaBoostClassifier(
        estimator=DecisionTreeClassifier(max_depth=3),
        n_estimators=50,
        learning_rate=1.0,
        random_state=RANDOM_STATE
    )
}

# -----------------------------
# Espacios de búsqueda de hiperparámetros
# -----------------------------
param_spaces = {
    'XGBoost': {
        'clf__n_estimators': stats.randint(50, 150),
        'clf__max_depth': stats.randint(3, 8),
        'clf__learning_rate': stats.uniform(0.01, 0.2),
        'clf__subsample': stats.uniform(0.7, 0.3),
        'clf__colsample_bytree': stats.uniform(0.7, 0.3),
        'clf__min_child_weight': stats.randint(1, 4)
    },
    'Gradient Boosting': {
        'clf__n_estimators': stats.randint(50, 150),
        'clf__learning_rate': stats.uniform(0.01, 0.2),
        'clf__max_depth': stats.randint(3, 6),
        'clf__subsample': stats.uniform(0.7, 0.3)
    },
    'AdaBoost': {
        'clf__n_estimators': stats.randint(25, 100),
        'clf__learning_rate': stats.uniform(0.01, 1.0)
    }
}

# -----------------------------
# Optimización de los top 3 modelos
# -----------------------------
optimized_models = {}

for model_name in top_3_models:  # lista de los top 3 modelos
    print(f"\nOPTIMIZANDO: {model_name}")

    base_estimator = estimators_base[model_name]

    # Crear pipeline: preprocesamiento + modelo
    pipe = Pipeline([
        ('preproc', preprocessor),
        ('clf', base_estimator)
    ])

    # RandomizedSearchCV
    rs = RandomizedSearchCV(
        estimator=pipe,
        param_distributions=param_spaces[model_name],
        n_iter=N_ITER_RS,
        scoring='f1',
        cv=CV,
        random_state=RANDOM_STATE,
        refit=True,
        n_jobs=-1,
        verbose=1
    )

    t0 = time.time()
    rs.fit(X_small, y_small)
    t1 = time.time()

    print(f"{model_name} optimizado en {(t1 - t0)/60:.2f} minutos")
    print(f"   Mejor F1 (CV): {rs.best_score_:.4f}")
    print(f"   Mejores parámetros: {rs.best_params_}")

    optimized_models[model_name] = rs.best_estimator_

# -----------------------------
# Evaluación rápida en la muestra
# -----------------------------
print("\nEvaluando desempeño en la muestra de entrenamiento reducida:")
scores_summary = []
for name, model in optimized_models.items():
    y_proba = model.predict_proba(X_small)[:, 1]
    auc = roc_auc_score(y_small, y_proba)
    scores_summary.append({'Modelo': name, 'ROC_AUC': auc})
    print(f"{name}: ROC_AUC = {auc:.4f}")

scores_df = pd.DataFrame(scores_summary).sort_values('ROC_AUC', ascending=False)

# -----------------------------
# Selección del mejor modelo
# -----------------------------
best_model_name = scores_df.iloc[0]['Modelo']
final_pipeline = optimized_models[best_model_name]
print(f"\nMejor modelo final: {best_model_name}")

# -----------------------------
# Reentrenamiento con todo el conjunto de entrenamiento
# -----------------------------
print("Reentrenando el mejor modelo con todo el conjunto de entrenamiento...")
final_pipeline.fit(X_train, y_train)
print("Reentrenamiento completo.")

# -----------------------------
# Guardar pipeline final
# -----------------------------
joblib.dump(final_pipeline, 'mejor_pipeline_opt.joblib')
print(f"Pipeline '{best_model_name}' guardado correctamente como 'mejor_pipeline_opt.joblib'.")

Usando muestra de 40939 registros para optimización rápida.


OPTIMIZANDO: XGBoost
Fitting 2 folds for each of 5 candidates, totalling 10 fits
XGBoost optimizado en 0.16 minutos
   Mejor F1 (CV): 0.8322
   Mejores parámetros: {'clf__colsample_bytree': np.float64(0.7139996989640846), 'clf__learning_rate': np.float64(0.20475110376829186), 'clf__max_depth': 5, 'clf__min_child_weight': 3, 'clf__n_estimators': 113, 'clf__subsample': np.float64(0.8400288679743939)}

OPTIMIZANDO: Gradient Boosting
Fitting 2 folds for each of 5 candidates, totalling 10 fits
Gradient Boosting optimizado en 0.95 minutos
   Mejor F1 (CV): 0.8364
   Mejores parámetros: {'clf__learning_rate': np.float64(0.12973169683940733), 'clf__max_depth': 5, 'clf__n_estimators': 132, 'clf__subsample': np.float64(0.7299924747454009)}

OPTIMIZANDO: AdaBoost
Fitting 2 folds for each of 5 candidates, totalling 10 fits
AdaBoost optimizado en 0.71 minutos
   Mejor F1 (CV): 0.8090
   Mejores parámetros: {'clf__learning_rate': np.float

Prueba de que guardo bien

In [35]:
# Cargar pipeline guardado
pipeline = joblib.load('mejor_pipeline_opt.joblib')

# Verificar columnas del dataset de prueba (usa X_test o una muestra del train)
X_test_sample = X_train.sample(10, random_state=42)  # solo una muestra pequeña
y_test_sample = y_train.loc[X_test_sample.index]

# Comprobar predict_proba
probas = pipeline.predict_proba(X_test_sample)[:, 1]
print("Probabilidades de la muestra:", probas)

# Revisar que no todas las probabilidades sean iguales
if np.all(probas == probas[0]):
    print(" ERROR: Todas las probabilidades son iguales")
else:
    print("Probabilidades variadas: OK")

# Comprobar predict
preds = pipeline.predict(X_test_sample)
print("Predicciones de la muestra:", preds)

# Validar tipos y cantidad de columnas
expected_cols = X_train.columns
if not all(col in X_test_sample.columns for col in expected_cols):
    print("ERROR: Faltan columnas en los datos de entrada")
else:
    print(" Columnas de entrada correctas")


Probabilidades de la muestra: [0.12246319 0.06666031 0.39435591 0.99197603 0.98172639 0.43927771
 0.12646433 0.29131529 0.06687801 0.64316251]
Probabilidades variadas: OK
Predicciones de la muestra: [0 0 0 1 1 0 0 0 0 1]
 Columnas de entrada correctas
